In [22]:
%reload_ext autoreload
%autoreload 2

import cv2
import numpy as np
import os
from aruco import detect_aruco, draw_aruco, select_aruco_poses, PoseSelectors
from segment_boxes import segment_and_draw_boxes_by_aruco
from utils import show
from calibrate_table import calibrate_table

In [3]:
dict_4x4 = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_1000)
params = cv2.aruco.DetectorParameters_create()
params.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_APRILTAG
params.perspectiveRemovePixelPerCell = 8
params.perspectiveRemoveIgnoredMarginPerCell = 0.26

In [29]:
calib = np.load('phitz/calib.npz')
K = calib['K']
D = calib['D']

In [30]:
folder = "phitz/table_markers/"
files = sorted(os.listdir(folder))
corners_3d_in_table_all = list()
for f in files:
    image = cv2.imread(folder + f)
    camera2table, corners_3d = calibrate_table(image, 0.132, K, D,
        dict_4x4, params)
    corners_3d = corners_3d[:, 0, 0, :]
    corners_3d = np.expand_dims(np.hstack((corners_3d, np.ones((4, 1)))), axis=-1)
    corners_3d_in_table = np.matmul(np.linalg.inv(camera2table), corners_3d)
    corners_3d_in_table = corners_3d_in_table[:, 0:3, 0]
    corners_3d_in_table_all.append(corners_3d_in_table)

corners_3d_in_table_all = np.array(corners_3d_in_table_all)


In [33]:
corners_3d_in_table_mean = corners_3d_in_table_all.mean(axis=0)
corners_3d_in_table_mean.shape
np.save('phitz/aruco_corners.npy', corners_3d_in_table_mean)